In [1]:
import numpy as np
import sqlite3
import pandas as pd
import csv
from datetime import datetime, timedelta
import time
import datetime
import requests
from io import StringIO




#抓取現在的時間
def take_time():
    from time import gmtime, strftime
    a=strftime("%Y%m%d", gmtime())
    return a[0:4],a[4:6],a[6:8]
    
    

# 抓上市公司股價
def stock_value(datestr):
    #把當月11號的台股資訊 更新
    #datestr = '20200911'
    # 下載股價
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + datestr + '&type=ALL')
    # 整理資料，變成表格
    df = pd.read_csv(StringIO(r.text.replace("=", "")), 
                header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
    # 整理一些字串：
    df = df.apply(lambda s: pd.to_numeric(s.astype(str).str.replace(",", "").replace("+", "1").replace("-", "-1"), errors='coerce'))
    # 顯示出來
    #df.head()
    return df




#抓上櫃公司股價
def stock_value2(datestr):
    link = 'http://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_download.php?l=zh-tw&d='+datestr+'&s=0,asc,0'
    r = requests.get(link)
    r.ok
    lines = r.text.replace('\r', '').split('\n')
    df = pd.read_csv(StringIO("\n".join(lines[3:])), header=None)
    #df.head()
    df.columns = list(map(lambda l: l.replace(' ',''), lines[2].split(',')))
    #df.index = df['代號']
    #df = df.drop(['代號'], axis=1)
    #df.head()
    return df

#拿上市收盤價資料
def take_value(df,id):
    take_info=df[pd.to_numeric(df['證券代號'], errors='coerce') == id]
    value=float(take_info["收盤價"].values)
    return value
#拿上櫃收盤價資料
def take_value2(df,id):
    take_info=df[pd.to_numeric(df['代號'], errors='coerce') == id]
    value=float(take_info["收盤"].values)
    return value

def change_parameter(save_list):
    final_update=now_day[:4]+"-"+now_day[4:6]+"-"+now_day[6:8]
    name=search_name(now_list[i][0]).replace("\u3000", " ")
    start_date=predict_day[:4]+"-"+predict_day[4:6]+"-"+predict_day[6:8]
    start_price=now_list[i][2]
    
    current_price=now_list[i][3]

    now_return=(float(current_price)-float(start_price))/float(start_price)
    now_return= round(now_return*100,2)

    if float(now_return)>0:
        types="+"
    else:
        types="-"
    return final_update,name,start_date,start_price,current_price,now_return,types

#查詢list in str
#any('1101' in item for item in a)
#查詢list in str
def search_name(name):
    company=[]
    df = pd.read_csv('公司/上市.csv')  
    a=df["有價證券代號及名稱"].tolist()
    df = pd.read_csv('公司/上櫃.csv')  
    b=df["有價證券代號及名稱"].tolist()
    df = pd.read_csv('公司/興櫃.csv')  
    c=df["有價證券代號及名稱"].tolist()
    for i in a:
        if i.__contains__(name) :
            company=i
            #print(i, " is containing")
    for i in b:
        if i.__contains__(name) :
            company=i
            #print(i, " is containing")
    for i in c:
        if i.__contains__(name) :
            company=i
            #print(i, " is containing")
    return company



with open('data.txt', 'r') as f:
    myNames = [line.strip() for line in f]
    
txt=myNames[0]+"-2"
#txt = input('請輸入你文件的名稱：')

## Open file   把當月份的代號txt  拿出來
doc_name=txt+".txt"
fp = open(doc_name, "r")
line = fp.readline()
month_predict=[]
## 用 while 逐行讀取檔案內容，直至檔案結尾
while line:
#    print (line)
    line=line.replace(" ","")
    month_predict.append(line.replace("\n",""))
    line = fp.readline()
fp.close()



nowyear,nowmonth,nowday=take_time()


# nowyear = input('請輸入今天的年份：')
# nowmonth = input('請輸入今天的月份：')
# nowday = input('請輸入今天的日期：')

predictyear=myNames[0][0:4]
predictmonth=myNames[0][4:6]

#predictyear = input('請輸入預測的年份：')
#predictmonth = input('請輸入預測的月份：')

predict_day=predictyear+predictmonth+"11"
now_day=nowyear+nowmonth+nowday


    

In [2]:
predict_day

'20210411'

In [3]:

#取出上市個股數值
for i in range(100):
    try:
        df_last_month=stock_value(predict_day)
        break
    except:
        
        from datetime import datetime
        edit_time=datetime.strptime(predict_day, "%Y%m%d")
        import datetime
        predict_day= (edit_time+datetime.timedelta(days=-1)).strftime("%Y%m%d")
        print(predict_day)
        
for i in range(100):
    try:
        df_now=stock_value(now_day)
        break
    except:
        from datetime import datetime
        edit_time=datetime.strptime(now_day, "%Y%m%d")
        import datetime
        now_day= (edit_time+datetime.timedelta(days=-1)).strftime("%Y%m%d")
        print(now_day)
        






nowyear2=str(int(nowyear)-1911)
predictyear2=str(int(predictyear)-1911)
predict_day2=predictyear2+"/"+predictmonth+"/"+"11"
now_day2=nowyear2+"/"+nowmonth+"/"+nowday


20210410
20210409


In [4]:


#取出上櫃個股數值
for i in range(100):
    try:
        print(predict_day2)
        df2_last_month=stock_value2(predict_day2)
        break
    except:
        from datetime import datetime
        predict_day_temp=str(int(predict_day2[0:3])+1911)+predict_day2[3:]
        edit_time=datetime.strptime(predict_day_temp, "%Y/%m/%d")
        import datetime
        predict_day_temp= (edit_time+datetime.timedelta(days=-1)).strftime("%Y/%m/%d")
        predict_day_temp_year=str(int(predict_day_temp[0:4])-1911)
        predict_day2=  predict_day_temp_year+predict_day_temp[4:]                                 
        print(predict_day2)                               
                                          
for i in range(100):
    try:
        print(now_day2)
        df2_now=stock_value2(now_day2)
        break
    except:
        from datetime import datetime
        now_day_temp=str(int(now_day2[0:3])+1911)+now_day2[3:]
        
        edit_time=datetime.strptime(now_day_temp, "%Y/%m/%d")
        import datetime
        now_day_temp= (edit_time+datetime.timedelta(days=-1)).strftime("%Y/%m/%d")
        now_day_temp_year=str(int(now_day_temp[0:4])-1911)
        now_day2=  now_day_temp_year+now_day_temp[4:]                                 
       
        print(now_day2)      



110/04/11
110/04/10
110/04/10
110/04/09
110/04/09
110/04/15


In [5]:

for i in range(len(month_predict)):

    try:
        df=df_now
        a=df[pd.to_numeric(df['證券代號'], errors='coerce') == float(month_predict[i])]
        b=a.values.tolist()[0]
        print(b[0])
    except:
        df=df2_now
        a=df[pd.to_numeric(df['代號'], errors='coerce') == float(month_predict[i])]
        b=a.values.tolist()[0]
        print(b[0])

1310.0
1326.0
1452.0
2106.0
2426.0
2606.0
2823.0
2834.0
2880.0
2887.0
2915.0
3455
4707
4904.0
6108.0
6207.0
8042
8415
8942


In [6]:








#把預測的股票價錢都拿出來
now_list=[]
for i in range(len(month_predict)):
    temp=[]
    index=float(month_predict[i])
    print(type(index),index)
    try:
        start_value=take_value(df_last_month,index)
        end_value=take_value(df_now,index)
    except:
        start_value=take_value2(df2_last_month,index)
        end_value=take_value2(df2_now,index)
    #print(i,str(index).replace(".0",""),start_value,end_value)
    temp.append(str(index).replace(".0",""))
    temp.append("")
    temp.append(start_value)
    temp.append(end_value)
    now_list.append(temp)





# #不要亂執行   拿來刪檔案用得
# db =  sqlite3.connect('db.sqlite3')
# ##db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
# db.execute("delete from basicHistory")
# db.commit()
# db =  sqlite3.connect('db.sqlite3')
# ##db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
# #db.execute("delete from technicHistory")
# db.commit()


over_date=predictyear+"-"+str(int(predictmonth)+1)+"-"+"11"


# 更新月營收策略  當月股價
# for i in range(len(now_list)):
#     final_update,name,start_date,start_price,current_price,now_return,types=change_parameter(now_list[i]) 
#     print(final_update,name,start_date,start_price,over_date,current_price,now_return,types)



<class 'float'> 1310.0
<class 'float'> 1326.0
<class 'float'> 1452.0
<class 'float'> 2106.0
<class 'float'> 2426.0
<class 'float'> 2606.0
<class 'float'> 2823.0
<class 'float'> 2834.0
<class 'float'> 2880.0
<class 'float'> 2887.0
<class 'float'> 2915.0
<class 'float'> 3455.0
<class 'float'> 4707.0
<class 'float'> 4904.0
<class 'float'> 6108.0
<class 'float'> 6207.0
<class 'float'> 8042.0
<class 'float'> 8415.0
<class 'float'> 8942.0


In [8]:




# 更新月營收策略  當月股價
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from technicCurrent")
db.commit()
pk=1
for i in range(len(now_list)):
    final_update,name,start_date,start_price,current_price,now_return,types=change_parameter(now_list[i]) 
    #print(predictmonth)
    if predictmonth=="12":
        predictmonth="01"  
        over_date=predictyear+"-"+str(int(predictmonth))+"-"+"11" 
    else:
        over_date=predictyear+"-"+str(int(predictmonth)+1)+"-"+"11" 
    print(final_update,name,start_date,start_price,over_date,current_price,now_return,types)
    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO technicCurrent (id,final_update,stock_name,start_date,start_price,over_date,current_price,now_return,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,final_update,name,start_date,start_price,over_date,current_price,now_return,types))
    db.commit()
    db.close()
    pk=pk+1

2021-04-15 1310 台苯 2021-04-09 20.55 2021-5-11 20.9 1.7 +
2021-04-15 1326 台化 2021-04-09 88.5 2021-5-11 90.1 1.81 +
2021-04-15 1452 宏益 2021-04-09 21.65 2021-5-11 21.8 0.69 +
2021-04-15 2106 建大 2021-04-09 42.5 2021-5-11 46.0 8.24 +
2021-04-15 2426 鼎元 2021-04-09 30.6 2021-5-11 29.15 -4.74 -
2021-04-15 2606 裕民 2021-04-09 41.85 2021-5-11 43.35 3.58 +
2021-04-15 2823 中壽 2021-04-09 25.1 2021-5-11 25.55 1.79 +
2021-04-15 2834 臺企銀 2021-04-09 9.77 2021-5-11 9.89 1.23 +
2021-04-15 2880 華南金 2021-04-09 18.55 2021-5-11 19.2 3.5 +
2021-04-15 2887 台新金 2021-04-09 13.3 2021-5-11 13.55 1.88 +
2021-04-15 2915 潤泰全 2021-04-09 78.6 2021-5-11 80.2 2.04 +
2021-04-15 3455 由田 2021-04-09 64.9 2021-5-11 66.0 1.69 +
2021-04-15 4707 磐亞 2021-04-09 12.9 2021-5-11 13.45 4.26 +
2021-04-15 4904 遠傳 2021-04-09 65.2 2021-5-11 65.6 0.61 +
2021-04-15 6108 競國 2021-04-09 22.25 2021-5-11 21.65 -2.7 -
2021-04-15 6207 雷科 2021-04-09 30.35 2021-5-11 29.68 -2.21 -
2021-04-15 8042 金山電 2021-04-09 61.9 2021-5-11 57.4 -7.27 -
2021-04-15 8